<a href="https://colab.research.google.com/github/ShivanshuPurohit/Practice-models/blob/master/Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/udacity/aind2-nlp-capstone/blob/master/data/small_vocab_en?raw=true
!wget https://github.com/udacity/aind2-nlp-capstone/blob/master/data/small_vocab_fr?raw=true

--2020-05-17 09:24:09--  https://github.com/udacity/aind2-nlp-capstone/blob/master/data/small_vocab_en?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/udacity/aind2-nlp-capstone/raw/master/data/small_vocab_en [following]
--2020-05-17 09:24:09--  https://github.com/udacity/aind2-nlp-capstone/raw/master/data/small_vocab_en
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/udacity/aind2-nlp-capstone/master/data/small_vocab_en [following]
--2020-05-17 09:24:10--  https://raw.githubusercontent.com/udacity/aind2-nlp-capstone/master/data/small_vocab_en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.

In [0]:
import os


def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r", encoding='utf-8') as f:
        data = f.read()

    return data.split('\n')

In [0]:
%load_ext autoreload
%autoreload 1

In [4]:
import collections
import numpy as np

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1206630308820650437
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13303772889712347525
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11161384669326527919
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11150726272
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16190803103595339390
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [6]:
# Add-on : Check framework versions being used 
import tensorflow as tf
import keras as K
import sys
print("Python", sys.version)
print("Tensorflow version", tf.__version__)
print("Keras version", K.__version__)

Python 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
Tensorflow version 2.2.0
Keras version 2.3.1


In [7]:
# Load English data
english_sentences = load_data('small_vocab_en')
# Load French data
french_sentences = load_data('small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


In [8]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [9]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [10]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk


# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [11]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        # Find the length of the longest sequence/sentence
        length = max([len(seq) for seq in x])
    
    return pad_sequences(sequences=x, maxlen=length, padding='post')


# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [12]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [13]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

# Add-on : Collect logs for Tensorboard
from keras.callbacks import TensorBoard
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=1, write_graph=True)

`logits_to_text` function loaded.


In [14]:
def nmt_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # TODO: Train the final model
    #print("Debug input_shape =" , input_shape, "  input_shape[1:] =", input_shape[1:], "  Input length=", input_shape[1:][0])
    #print("Debug output_sequence_length =" , output_sequence_length) 
    #print("Debug english_vocab_size =" , english_vocab_size)
    #print("Debug french_vocab_size =" , french_vocab_size)
    
    
    ## Model_final is a seq2seq model (encoder-decoder) using embedding and bidirectional LSTM
    
    # Hyperparameters
    embedding_size = 256
    rnn_cells = 300
    dropout = 0.2
    learning_rate = 1e-3
    
    from keras.layers import LSTM, concatenate
    
    # Input and embedding
    encoder_input_seq = Input(shape=input_shape[1:]) 
    embedded_input_seq = Embedding(input_dim = english_vocab_size,
                                   output_dim = embedding_size,
                                   input_length=input_shape[1:][0])(encoder_input_seq)
    
    ## Note Bidirectional LSTM is used on the encoder side (see https://arxiv.org/pdf/1609.08144.pdf )
    ## Alternate version : Encoder RNN Bidirectional layer (Using the Keras Bidirectional layer wrappers)
    #encoder_output, forward_state_h, forward_state_c, backward_state_h, backward_state_c = Bidirectional(LSTM(units=rnn_cells,
    #                                                                                                          dropout=dropout,
    #                                                                                                          return_sequences=False,
    #                                                                                                          return_state=True))(embedded_input_seq)
    
    # Encoder Forward RNN layer
    encoder_forward_output, forward_state_h, forward_state_c = LSTM(units=rnn_cells,
                                                                    dropout=dropout,
                                                                    return_sequences=False,
                                                                    return_state=True,
                                                                    go_backwards=False)(embedded_input_seq)
    
    # Encoder backward RNN layer
    encoder_backward_output, backward_state_h, backward_state_c = LSTM(units=rnn_cells,
                                                                       dropout=dropout,
                                                                       return_sequences=False,
                                                                       return_state=True,
                                                                       go_backwards=True)(embedded_input_seq)
    
    # Encoder output and states : Merge the LSTM Forward and Backward ouputs (using 'concatenate' method) 
    state_h = concatenate([forward_state_h, backward_state_h]) 
    state_c = concatenate([forward_state_c, backward_state_c])    
    encoder_output = concatenate([encoder_forward_output, encoder_backward_output])        
         
    # Decoder Input   
    decoder_input_seq = RepeatVector(output_sequence_length)(encoder_output)
    
    # Decoder RNN layer
    # Note : we need twice more LSTM cells as we have concatenated backward and forward LSTM layers in encoder
    decoder_output = LSTM(units=rnn_cells*2,
                                  dropout=dropout,
                                  return_sequences=True,
                                  return_state=False,
                                  go_backwards=False)(decoder_input_seq, initial_state=[state_h, state_c])
    
    # Decoder output
    logits = TimeDistributed(Dense(units=french_vocab_size))(decoder_output) 
    
    # Model
    model = Model(encoder_input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    
    return model    
       

# Train and Print prediction(s)

# Pad the input to work with the Embedding layer
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
#print("Debug tmp_x shape=", tmp_x.shape )


# Train the neural network 
translation_model = nmt_model(input_shape = tmp_x.shape,
                              output_sequence_length = max_french_sequence_length,
                              english_vocab_size = english_vocab_size+1,
                              french_vocab_size = french_vocab_size+1)

print(translation_model.summary(line_length=125))


translation_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=15, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(nmt_model.predict(tmp_x[:1])[0:10], french_tokenizer))

Model: "model_1"
_____________________________________________________________________________________________________________________________
Layer (type)                             Output Shape               Param #        Connected to                              
input_1 (InputLayer)                     (None, 21)                 0                                                        
_____________________________________________________________________________________________________________________________
embedding_1 (Embedding)                  (None, 21, 256)            51200          input_1[0][0]                             
_____________________________________________________________________________________________________________________________
lstm_1 (LSTM)                            [(None, 300), (None, 300), 668400         embedding_1[0][0]                         
_____________________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/15
110288/110288 [==============================] - 80s 723us/step - loss: 2.4890 - accuracy: 0.4875 - val_loss: 1.8058 - val_accuracy: 0.5365
Epoch 2/15
110288/110288 [==============================] - 78s 710us/step - loss: 1.4530 - accuracy: 0.6129 - val_loss: 1.2571 - val_accuracy: 0.6610
Epoch 3/15
110288/110288 [==============================] - 79s 712us/step - loss: 1.1502 - accuracy: 0.6825 - val_loss: 0.9862 - val_accuracy: 0.7194
Epoch 4/15
110288/110288 [==============================] - 78s 708us/step - loss: 0.9292 - accuracy: 0.7306 - val_loss: 0.8264 - val_accuracy: 0.7565
Epoch 5/15
110288/110288 [==============================] - 78s 710us/step - loss: 0.7613 - accuracy: 0.7732 - val_loss: 0.6587 - val_accuracy: 0.8016
Epoch 6/15
110288/110288 [==============================] - 78s 711us/step - loss: 0.6302 - accuracy: 0.8087 - val_loss: 0.5404 - val_accuracy: 0.8355
Epoch 7/15
110288/110288 [=================

AttributeError: ignored

In [15]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = nmt_model(input_shape = x.shape,
                        output_sequence_length = y.shape[1],
                        english_vocab_size = len(x_tk.word_index)+1,
                        french_vocab_size = len(y_tk.word_index)+1)

    model.fit(x, y, batch_size=1024, epochs=20, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 71s 643us/step - loss: 2.6654 - accuracy: 0.4698 - val_loss: 1.7557 - val_accuracy: 0.5616
Epoch 2/20
110288/110288 [==============================] - 70s 633us/step - loss: 1.4816 - accuracy: 0.6097 - val_loss: 1.2382 - val_accuracy: 0.6601
Epoch 3/20
110288/110288 [==============================] - 70s 635us/step - loss: 1.1345 - accuracy: 0.6840 - val_loss: 0.9978 - val_accuracy: 0.7166
Epoch 4/20
110288/110288 [==============================] - 70s 635us/step - loss: 0.9449 - accuracy: 0.7262 - val_loss: 0.8483 - val_accuracy: 0.7499
Epoch 5/20
110288/110288 [==============================] - 70s 635us/step - loss: 0.7938 - accuracy: 0.7636 - val_loss: 0.6899 - val_accuracy: 0.7928
Epoch 6/20
110288/110288 [==============================] - 70s 634us/step - loss: 0.6535 - accuracy: 0.8025 - val_loss: 0.5652 - val_accuracy: 0.8290
Epoch 7/20
110288/110288 [=================

In [16]:
print(logits_to_text(nmt_model.predict(tmp_x[5:10])[0], french_tokenizer))

AttributeError: ignored